* Importations des librairies courantes

In [ ]:
# Directive pour afficher les graphiques dans Jupyter (inutile si on utilise Spyder)
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from pandas import read_csv
#Chargeant les données
data = pd.read_csv('../input/bostonhoustingmlnd/housing.csv')

In [ ]:
# Simulant les données
data.head()

In [ ]:
# collecte d'infos 
data.info()


Le problème que nous allons résoudre ici est sur la base d'un ensemble de fonctionnalités décrivant une maison à Boston, notre modèle d'apprentissage automatique doit prédire le prix de l'immobilier. Pour former notre modèle d'apprentissage automatique avec les données de logement à Boston,on spécifie
* RM - Nombre moyen de pièces par logement
* LSTAT -  % de statut inférieur de la population
* PTRATIO - ratio élèves-enseignants par ville
* MEDV - Valeur médiane des maisons occupées par leur propriétaire en 1000 $
sur la base des ces paramétres on cherche à prédire le prix d'immobilier à BOSTON

In [ ]:
# comptage
data.count()

In [ ]:
## Dimension de la Data set ( ligne , colonne)
print(np.shape(data))

In [ ]:
## Résumons les données pour voir la distribution des données
print(data.describe())

In [ ]:
data.plot(kind="scatter", x="LSTAT", y="RM", c="MEDV", cmap="rainbow", s=3, figsize=(12,12))

Effectuant une condition ou MEDV<500000 et s'aperçois qu'il a plus de rouge 

In [ ]:
data[data.MEDV<500000].plot(kind="scatter", x="LSTAT", y="RM", c="MEDV", cmap="rainbow", s=3, figsize=(12,12))

In [ ]:
data[data.MEDV<500000].groupby(['LSTAT','RM'])['MEDV'].mean().plot(kind = 'bar', figsize=(50,20))

On detecte une fluctuation de la valeur MEDV

Tracons l'ensemble de données et voyons les tendances / statistiques intéressantes.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
data = pd.read_csv('../input/bostonhoustingmlnd/housing.csv')
fig, axs = plt.subplots(ncols=4, nrows=1, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in data.items():
    sns.boxplot(y=k, data=data, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
tabcorr = data.corr()     # on peut utiliser aussi bos.corr(method='pearson') par exemple

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(abs(tabcorr), cmap="coolwarm",annot=True)


Voyons à quoi ressemblent ces fonctionnalités et les distributions MEDV
* MEDV semble également avoir une distribution normale (les prédictions) et d'autres colonnes semblent avoir une distribution norma ou bimodèle des données 
* On va utiliser la fonction corr pour calculer systématiquement le degré de corrélation entre deux paramètres:
* Pour visualiser l'ensemble du tableau de corrélations, on utilise une "carte de températures" (heatmap) :
* À partir de la matrice de corrélation, nous voyons. Les colonnes LSTAT,RM,PTRAIO ont un score de corrélation supérieur à 0,5 avec MEDV, ce qui est une bonne indication de l'utilisation comme prédicteurs. Traçons ces colonnes par rapport à MEDV.

On peut aussi regrouper les paramètres par clusters classés par proximité :

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm", annot=True)

On peut tracer uniquement le dendrogramme des corrélations entre les caractéristiques. On utilise une forme "condensée" de la matrice de corrélation. linkage permet de calculer les distances entre caractéristiques à partir des corrélations

In [ ]:
from scipy.cluster import hierarchy as hc
corr = 1 - data.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=data.columns, orientation='left', leaf_font_size=10)

In [ ]:
correlations = tabcorr.RM
print(correlations)

In [ ]:
print(abs(correlations).sort_values(ascending=False))

**Régression linéaire multiple**


Essayons d'abord la régression linéaire sur l'ensemble de données.

In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop(['MEDV'], axis=1)
y = data.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

On utilise la fonction de régression linéaire multiple de sklearn :

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)            # apprentissage
y_pred = lm.predict(X_test)         # prédiction sur l'ensemble de test

In [ ]:
#On trace le nuage de points pour comparer la prédiction et les résultats attendus :
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction de MEDV")
plt.title("MEDV reels vs predictions")

In [ ]:
#ou on peut visualiser la distribution de l'erreur avec seaborn :
sns.distplot(y_test-y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
#On peut calculer l'erreur sur les moindres carrés :
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
#Ou le score R2 (rapport des variances estimée/réelle) :
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

In [ ]:
#qu'on peut écrire plus simplement :
lm.score(X_test,y_test)

Régression par forêts aléatoires

In [ ]:
#On va utiliser les forêts aléatoires pour la régression. On conserve l'ensemble des caractéristiques
X = data.drop(['MEDV'], axis=1)
y = data.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction de MEDV")
plt.title("MEDV reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction de MEDV")
plt.title("MEDV reels vs predictions")